In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import time
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import numpy as np
import datetime
from selenium.webdriver.common.action_chains import ActionChains
import xlsxwriter
from openpyxl import Workbook, load_workbook
import os
import shutil
import logging
import traceback
import re
import csv
from azure.storage.blob import BlobServiceClient
import argparse
import sys
import glob
import calendar
import json

In [2]:

# File paths


# date_today = datetime.date.today().strftime("%Y-%m-%d")
# output_musement = r'output/GYG'
# archive_folder = fr'{output_musement}/Archive'
# file_path_done =fr'output/GYG/{date_today}-DONE-GYG.csv'  
# file_path_output = fr"output/GYG - {date_today}.xlsx"
# link_file = fr'resource/GYG_links.csv'
# avg_file = fr'resource/avg-gyg.csv'
# re_run_path = fr'output/GYG/{date_today}-ReRun-GYG.csv'
# folder_path_with_txt_to_count_avg = 'Avg/GYG'
def constant_file_path():

    global output_musement, archive_folder, link_file, logs_path, storage_account_name, storage_account_key, container_name_raw, container_name_refined,  link_file_path, site_name
    site_name = "Musement"
    output_musement = fr'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/{site_name}'
    archive_folder = fr'{output_musement}/Archive'
    link_file = fr'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Resource/{site_name}_links.csv'
    logs_path = fr'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Logs/{site_name}'

    # Set the name of your Azure Storage account and the corresponding access key
    storage_account_name = "storagemyotas"
    storage_account_key = "vyHHUXSN761ELqivtl/U3F61lUY27jGrLIKOyAplmE0krUzwaJuFVomDXsIc51ZkFWMjtxZ8wJiN+AStbsJHjA=="

    # Set the name of the container and the desired blob name
    container_name_raw = f"raw/daily/{site_name.lower()}"
    container_name_refined = f"refined/daily/{site_name.lower()}"
    

def configure_dates_and_file_names():
    global date_today, extraction_date, extraction_date_save_format, local_file_path, blob_name,  file_path_logs_processed, output_file_path
    date_today = datetime.date.today().strftime("%Y-%m-%d")
    # Format the date and time as a string
    extraction_date = datetime.datetime.now().strftime('%Y-%m-%d %H:00:00')
    # extraction_date_save_format = f"{extraction_date.replace(' ', '_').replace(':','-')}_{language}_{adutls}"
    # Set the path of the local file

    local_file_path = f"{output_musement}/{site_name} - {date_today}.xlsx"
    blob_name = fr'{site_name} - {date_today}.xlsx'
    file_path_logs_processed = fr'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Logs/files_processed/{blob_name.split(".")[0]}'

constant_file_path()
configure_dates_and_file_names()

In [3]:
# create logger object
logger_err = logging.getLogger('Error_logger')
logger_err.setLevel(logging.DEBUG)
logger_info = logging.getLogger('Info_logger')
logger_info.setLevel(logging.DEBUG)
logger_done = logging.getLogger('Done_logger')
logger_done.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create file handler for error logs and set level to debug
fh_error = logging.FileHandler(fr'{logs_path}/error_logs.log')
fh_error.setLevel(logging.DEBUG)

# create file handler for info logs and set level to info
fh_info = logging.FileHandler(fr'{logs_path}/info_logs.log')
fh_info.setLevel(logging.INFO)

# create file handler for info logs and set level to info
fh_done = logging.FileHandler(fr'{logs_path}/done_logs.log')
fh_done.setLevel(logging.INFO)
# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to handlers
ch.setFormatter(formatter)
fh_error.setFormatter(formatter)
fh_info.setFormatter(formatter)
fh_done.setFormatter(formatter)

# add handlers to logger
logger_err.addHandler(ch)
logger_err.addHandler(fh_error)
logger_info.addHandler(ch)
logger_info.addHandler(fh_info)
logger_done.addHandler(ch)
logger_done.addHandler(fh_done)

In [4]:
def handle_error_and_rerun(error):
#     recipient_error = 'wojbal3@gmail.com'
    tb = traceback.format_exc()
    logger_err.error('An error occurred: {} on {}'.format(str(error), tb))
#     subject = f'Error occurred - {time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())}'
#     message = f'<html><body><p>Error occurred: {str(error)} on {tb}</p></body></html>'
#     send_email(subject, message, recipient_error)

In [5]:
def combine_csv_to_xlsx():
    """
    This function combines all CSV files in the specified output directory that have
    a filename starting with today's date into a single Excel file.
    Each CSV file is written as a separate sheet in the Excel file.
    After combining, the original CSV files are moved to the archive folder.
    """
    date_today = '2024-03-32'
    # Get all CSV files with the specified date prefix in the output directory
    csv_files = [file for file in os.listdir(output_gyg) if file.endswith('.csv') and file.startswith(date_today)]


    # Check if no CSV files were found and exit the function if true
    if not csv_files:
        print(f"No CSV files found with the date prefix '{date_today}'")
        return

    # Specify the output Excel file path and name
    output_file = f"{output_gyg}/GYG - {date_today}.xlsx"
    # Create a Pandas Excel writer using XlsxWriter as the engine
    writer = pd.ExcelWriter(output_file, engine='xlsxwriter')
    
    for csv_file in csv_files:
        # Construct the full file path for the CSV file
        csv_path = os.path.join(output_gyg, csv_file)
        
        # Generate a sheet name based on the CSV file name
        sheet_name = os.path.splitext(csv_file)[0]
        sheet_name = sheet_name.split(date_today + '-')[1].split('-GYG')[0]
        
        # Read the CSV file into a Pandas DataFrame
        df = pd.read_csv(csv_path)
        
        # Write the DataFrame to the Excel file as a new sheet
        df.to_excel(writer, sheet_name=sheet_name, index=False)

    # Save and close the Excel writer to finalize the Excel file
    writer.close()

    # Log the successful combination of CSV files
    print(f"Combined CSV files with date prefix '{date_today}' into '{output_file}'")

    # Move the original CSV files to the Archive folder
    for csv_file in csv_files:
        # Construct the full file path for the CSV file
        csv_path = os.path.join(output_gyg, csv_file)
        # Specify the destination path in the archive folder
        destination_path = os.path.join(archive_folder, csv_file)
        # Move the CSV file to the Archive folder
        shutil.move(csv_path, destination_path)

    # Log the successful archival of CSV files
    print(f"Moved {len(csv_files)} CSV file(s) to the '{archive_folder}' folder.")

In [6]:
def create_log_done(log_type):
    global file_path_logs_processed
    if log_type == 'Raw':
        with open(f'{file_path_logs_processed}-raw.txt', 'w') as file:
            file.write('Done')
    elif log_type == 'Refined':
        with open(f'{file_path_logs_processed}-refined.txt', 'w') as file:
            file.write('Done')

In [7]:
def upload_excel_to_azure_storage_account(local_file_path, storage_account_name, storage_account_key, container_name_raw, blob_name):
    try:
        # Create a connection string to the Azure Storage account
        connection_string = f"DefaultEndpointsProtocol=https;AccountName={storage_account_name};AccountKey={storage_account_key};EndpointSuffix=core.windows.net"

        # Create a BlobServiceClient object using the connection string
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)

        # Get a reference to the container
        container_client = blob_service_client.get_container_client(container_name_raw)

        # Upload the file to Azure Blob Storage
        with open(local_file_path, "rb") as file:
            container_client.upload_blob(name=blob_name, data=file)
        create_log_done('Raw')
        print("File uploaded successfully to Azure Blob Storage (raw).")

    except Exception as e:
        print(f"An error occurred: {e}")

In [8]:
def transform_upload_to_refined(local_file_path, storage_account_name, storage_account_key, container_name_refined, blob_name):
    exclude_sheets = ['Sheet1', 'Data', 'Re-Run', 'DONE']
    # Define the Azure Blob Storage connection details
    connection_string = f"DefaultEndpointsProtocol=https;AccountName={storage_account_name};AccountKey={storage_account_key};EndpointSuffix=core.windows.net"
    # Read the Excel file into a Pandas DataFrame
    excel_data = pd.read_excel(local_file_path, sheet_name=None)
    output_file_path = "temp_file.xlsx"
    with pd.ExcelWriter(output_file_path) as writer:
        for sheet_name, df in excel_data.items():
            if sheet_name in exclude_sheets:
                continue
            # Make changes to the df DataFrame as needed
            df['Data zestawienia'] = df['Data zestawienia'].astype('str')
            df['IloscOpini'].fillna(0, inplace= True)
            df['Opinia'].fillna('N/A', inplace=True)
            df = df[df['Tytul'] != 'Tytul']
            df = df[df['Data zestawienia'] != 'Data zestawienia']
            df = df[df['Data zestawienia'].str.len() > 4]
            df.drop(columns=['VPN_City', 'Tekst'], inplace=True)
            df['Booked'] = df['Booked'].astype('str')
            df['Przecena'] = df['Przecena'].astype('str')
            df['Cena'] = df['Cena'].map(lambda x: x.split(x[0])[1].strip() if not x[0].isnumeric() else x)
            df['Booked'] = df['Booked'].str.replace('New activity', 'nan')
            df['Booked'] = df['Booked'].map(lambda x: x.split('Booked')[1].split()[0] if len(x) > 5 else x)
            df['Przecena'] = df['Przecena'].str.replace(r'[$€£]', '', regex=True).str.strip()
            df['Przecena'] = df['Przecena'].map(lambda x: x.split()[0] if len(x) > 4 else x)
            df['Przecena'].fillna("NULL", inplace= True)
            #     df['VPN_City'].fillna("NULL", inplace= True)
            df['Booked'].fillna("NULL", inplace= True)
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    # Create a connection to Azure Blob Storage
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    container_client = blob_service_client.get_container_client(container_name_refined)

    # Upload the modified Excel file to Azure Blob Storage
    with open(output_file_path, "rb") as data:
        container_client.upload_blob(name=blob_name, data=data)
        
    print("File uploaded successfully to Azure Blob Storage (refined).")
    os.remove(output_file_path)
    create_log_done('Refined')
    return 'Added to Blob'


In [18]:
df_links = pd.read_csv(link_file)
df_links = df_links[(df_links['Category']=='Global') & (df_links['City'] == "Lisbon")]

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

In [19]:
start_time = time.time()
total_pages = 0
iter = 0
for index, row in df_links.iterrows():
    
#     CHECK IF FILE PATH EXISIT IF SO CHECK THE DATA INSIDE
#         print(index, row)
    page = 1
    max_pages = 9999
    data = []
    position = 0
    url_time = time.time()
    driver.get(row['URL'])

    

In [21]:
currency_element = driver.find_element("css selector", "div[data-test='footer-dropdown-currency']")
driver.execute_script("arguments[0].scrollIntoView(true);", currency_element)
currency_element.click()

In [ ]:
# def daily_run_viator(df_links=pd.DataFrame(), re_run=False):

df_links=pd.DataFrame()

if len(df_links) == 0:
    df_links = pd.read_csv(link_file)
    df_links = df_links.head(20)

EUR_City = [
    "Amsterdam", "Athens", "Barcelona", "Berlin", "Dublin", "Dubrovnik", "Florence", "Istanbul",
    "Krakow", "Lisbon", "Madrid", "Milan", "Naples", "Paris", "Porto", "Rome", "Palermo", "Venice",
    "Taormina", "Capri", "Sorrento", "Mount-Etna", "Mount-Vesuvius", "Herculaneum", "Amalfi-Coast",
    "Pompeii"
]

USD_City = [
    "Las-Vegas", "New-York-City", "Cancun", "Dubai"
]

GBP_City = [
    "Edinburgh", "London"
]

#     date_today = datetime.date.today().strftime("%Y-%m-%d")
#     file_path_done =fr'output/Viator/{date_today}-DONE-Viator.csv'  
#     file_path_output = f"output/Viator - {date_today}.xlsx"
if os.path.exists(file_path_output) and re_run == False:
    print(f'Today ({date_today}) Viator done')
#     return 'Done'



if os.path.exists(file_path_done) and re_run == False:
    done_msg = pd.read_csv(file_path_done)
    done_msg = done_msg.transpose()
    done_msg = done_msg.set_axis(done_msg.iloc[0], axis=1)
    done_msg = done_msg.iloc[1:]
    done_index = int(done_msg.index.values[0])
    df_links = df_links.iloc[(done_index+1):]
#         display(df_links)
elif re_run == True:
    print(f'Lenght of links: {len(df_links)}')
else:
    print("Nothing done yet")
options = webdriver.ChromeOptions()
#     options.add_argument('--blink-settings=imagesEnabled=false')
#     options.add_argument("--disable-gpu")
#     options.add_argument("--no-sandbox")
#     options.add_argument("--log-level=3")
options.add_argument("--disable-infobars")
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches",["enable-automation"])
#     options.add_argument("--user-data-dir=C:/Users/Wojciech/AppData/Local/Google/Chrome/User Data/Default")
#     options.add_argument('start-maximized')
#     options.add_argument('disable-infobars')
#     options.add_argument("--disable-logging")
#     options.add_argument("--disable-login-animations")
#     options.add_argument("--disable-notifications")
#     options.add_argument("--disable-default-apps")
#     options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.maximize_window()
# Define the URL of the website we want to scrape
start_time = time.time()
total_pages = 0
max_to_do = 0
if len(df_links) == 0:
#     return 'Done'
    pass
for index, row in df_links.iterrows():
#     CHECK IF FILE PATH EXISIT IF SO CHECK THE DATA INSIDE
    print(row["URL"])

    data = []
    position = 0
    url_time = time.time()
    url = f'{row["URL"]}'
    driver.get(url)
#         print(url)
    currency = driver.find_element(By.CSS_SELECTOR, ("[data-type=currency]")).text.strip()
#         currency = soup.select_one("[data-type=currency]").text.strip()

    if row['City'] in EUR_City:
        if '€' in currency:
            pass
        else:
            # Assuming you have initialized the webdriver and assigned it to the 'driver' variable
            button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='EUR']")
            # Scroll to the element not requried
            # driver.execute_script("arguments[0].scrollIntoView();", element)
            # Click the element
            driver.execute_script("arguments[0].click();", button)

    elif row['City'] in USD_City:
        if '$' in currency:
            pass
        else:
            # Assuming you have initialized the webdriver and assigned it to the 'driver' variable
            button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='USD']")
            # Scroll to the element not requried
            # driver.execute_script("arguments[0].scrollIntoView();", element)
            # Click the element
            driver.execute_script("arguments[0].click();", button)
    elif row['City'] in GBP_City:
        if '£' in currency:
            pass
        else:
            # Assuming you have initialized the webdriver and assigned it to the 'driver' variable
            button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='GBP']")
            # Scroll to the element not requried
            # driver.execute_script("arguments[0].scrollIntoView();", element)
            # Click the element
            driver.execute_script("arguments[0].click();", button)

    else:
        print('Missing from the list:', city)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    try:
        product_list_count = int(soup.select_one("[data-test-id=activity-count-text]").text.strip().split()[0])
        max_pages = int(round(int(product_list_count)/12,0) + 1)
    except:
        print('Max page cannot be found')
    driver.get(url)
    time.sleep(1)
    products = ''
    print('product_list_count:', product_list_count)
    while len(products) < product_list_count:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        products = soup.select("[data-test=ActivityCard]")
        print(len(products))
        try:
            more_activities = driver.find_element(By.CSS_SELECTOR, 'button[data-test="test-btn"]')
            driver.execute_script("arguments[0].click();", more_activities)
            time.sleep(1)
        except:
            print('E')
            time.sleep(2)
            
    print('Products check')
    for product in products:
        try:
            title = product.find('a').text.strip() # Title
        except:
            print(f'For url {url} collected {position}')
            break
        product_url = product.find('a')['href'] #URL
        price = product.select_one("[data-test=realPrice]").text # Price
        position = position + 1
        siteuse = 'Musement'
        city = row['City']
        category = row['Category']
        try:
            stars = product.select_one("[data-test=reviewTest]").find('strong').text.strip() # Ratings
        except:
            stars = 'N/A'
        try:
            amount_reviews = product.select_one("p[class*=reviewsNumber]").text.replace('(','').replace(')','') # Amount of reviews
        except:
            amount_reviews = 'N/A'
        
        data.append([title,product_url, price, stars, amount_reviews, date_today, position, category, siteuse, city ])
        
    url_done = time.time()
    message = f'Time for {city}-{category}: {round((url_done - url_time)/60, 3)}min | Pages: {max_pages} | AVG {round((url_done - url_time)/max_pages, 2)}s per page Currency: 1-{first_style_curr}, 2-{second_style_curr}, 3-{thirtd_style_curr}'
    print(message)
    logger_info.info(message)
    df = pd.DataFrame(data, columns=['Tytul', 'Tytul URL', 'Cena', 'Opinia', 'IloscOpini', 'Data zestawienia', 'Pozycja', 'Kategoria', 'SiteUse', 'Miasto'])
    
    file_path = fr'{output_musement}/{date_today}-{city}-Musement.csv' 
    df.to_csv(file_path, header=not os.path.exists(file_path), index=False, mode='a')
    row.to_csv(file_path_done, header=True, index=True)    

Nothing done yet
https://www.musement.com/uk/amsterdam/?sort_by=-relevance-city&limit=12
product_list_count: 363
12
24
36
48
48
48
0
12
24
36
48
48
48
48
48
12
24
36
0
12
24
24
12
24
24
24
12
24
36
48
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E

In [60]:
df = pd.DataFrame(data, columns=['Tytul', 'Tytul URL', 'Cena', 'Opinia', 'IloscOpini', 'Data zestawienia', 'Pozycja', 'Kategoria', 'SiteUse', 'Miasto'])
df

,Tytul,Tytul URL,Cena,Opinia,IloscOpini,Data zestawienia,Pozycja,Kategoria,SiteUse,Miasto
0,Van Gogh Museum ticket,/uk/amsterdam/van-gogh-museum-ticket-651/,€22.00,4.4,6467,2023-06-28,1,Global,Musement,Amsterdam
1,ARTIS Amsterdam Royal Zoo entrance ticket,/uk/amsterdam/artis-amsterdam-royal-zoo-entran...,€25.00,4.7,77,2023-06-28,2,Global,Musement,Amsterdam
2,Rijksmuseum entrance ticket,/uk/amsterdam/rijksmuseum-entrance-ticket-91921/,€23.50,4.7,303,2023-06-28,3,Global,Musement,Amsterdam
3,Heineken® Experience tickets,/uk/amsterdam/heineken-r-experience-tickets-2224/,€21.00,4.5,350,2023-06-28,4,Global,Musement,Amsterdam
4,Ticket to AMAZE immersive audiovisual experience,/uk/amsterdam/ticket-to-amaze-immersive-audiov...,€26.95,4.6,5,2023-06-28,5,Global,Musement,Amsterdam
...,...,...,...,...,...,...,...,...,...,...
355,Europe audio guide with TravelMate app,/uk/rome/europe-audio-guide-with-travelmate-ap...,€36.90,N/A,N/A,2023-06-28,356,Global,Musement,Amsterdam
356,The Netherlands private day trip to the Dutch ...,/uk/amsterdam/the-netherlands-private-day-trip...,€590.00,N/A,N/A,2023-06-28,357,Global,Musement,Amsterdam
357,The Story of Edam Cheese museum entrance ticket,/uk/volendam/the-story-of-edam-cheese-museum-e...,€8.00,5,1,2023-06-28,358,Global,Musement,Amsterdam
358,Holland four cities private day-trip,/uk/amsterdam/holland-four-cities-private-day-...,€110.00,N/A,N/A,2023-06-28,359,Global,Musement,Amsterdam


In [67]:
product = products[208]
product_url = product.find('a')['href'] #URL
stars = product.select_one("[data-test=reviewTest]").find('strong').text.strip() # Ratings
amount_reviews = product.select_one("p[class*=reviewsNumber]").text.replace('(','').replace(')','') # Amount of reviews
price = product.select_one("[data-test=realPrice]").text # Price
title = product.find('a').text.strip() # Title

# data.append([title,product_url, price, stars, amount_reviews, discount, text, date_today, position, category, siteuse, city ])

AttributeError: 'NoneType' object has no attribute 'find'

In [64]:
title

'2-hour small-group Rijksmuseum guided tour'